In [15]:
import Pkg; Pkg.activate("..")

  Activating project at `~/SAFT_ML`


In [16]:
using CSV, DataFrames, DelimitedFiles, Clapeyron, PyCall
import PyPlot; const plt = PyPlot ;

In [17]:
df = CSV.read("SI_pcp-saft_parameters.csv",DataFrame,header=1) ;
fieldnames(typeof(df))
#display(names(df)[1:20]) ;
writedlm("out.txt", names(df), ' ') ;

replace!(df.mu, missing => 0) ;
replace!(df.kappa_ab, missing => 0) ;
replace!(df.epsilon_k_ab, missing => 0) ;

df.mu = Float64.(df.mu)
df.kappa_ab = Float64.(df.kappa_ab)
df.epsilon_k_ab = Float64.(df.epsilon_k_ab) ;
df

Row,common_name,iupac_name,inchi,canonical_smiles,isomeric_smiles,cas,family,molarweight,m,sigma,epsilon_k,mu,kappa_ab,epsilon_k_ab,na,nb,mard_psat,t_min_psat,t_max_psat,points_psat,mard_psat_incl_outlier,points_psat_incl_outlier,mard_density,t_min_density,t_max_density,points_density_liquid,mard_density_liquid_single_phase,points_density_liquid_single_phase,mard_density_equi,points_density_equi,mard_density_vapor,points_density_vapor,opt,bounds_violation
,String,String?,String,String,String,String15,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Float64,Int64,Float64,Int64,Float64,Float64,Float64,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,String15,String
1,acetaldehyde,acetaldehyde,"InChI=1S/C2H4O/c1-2-3/h2H,1H3",CC=O,CC=O,75-07-0,Aldehydes,44.026,2.14028,3.22662,227.621,2.69,0.0,0.0,1,0,2.84,225.15,382.9,197,4.5,207,0.28,253.15,323.15,87,0.27,74.0,0.31,13.0,missing,missing,DIPOLAR,OK
2,acetamide,acetamide,"InChI=1S/C2H5NO/c1-2(3)4/h1H3,(H2,3,4)",CC(=N)O,CC(=N)O,60-35-5,Amines,59.037,2.82648,3.12045,377.672,0.0,0.01458,1624.89,1,2,1.35,355.59,495.15,24,1.9,25,1.04,353.15,463.5,19,1.04,19.0,missing,missing,missing,missing,ASSOCIATIVE,OK
3,acetonitrile,acetonitrile,InChI=1S/C2H3N/c1-2-3/h1H3,CC#N,CC#N,75-05-8,Nitriles,41.027,2.32567,3.16425,207.704,3.92,0.0,0.0,1,0,1.98,273.15,535.032,846,2.18,852,1.45,228.15,513.26,1426,1.47,1377.0,0.79,49.0,6.63,106.0,DIPOLAR,OK
4,acetone,propan-2-one,InChI=1S/C3H6O/c1-3(2)4/h1-2H3,CC(C)=O,CC(C)=O,67-64-1,Ketones,58.042,2.78618,3.24884,231.118,2.88,0.0,0.0,1,0,0.96,197.15,508.65,1619,2.41,1666,0.63,178.0,480.86,1943,0.63,1906.0,0.63,37.0,4.06,443.0,DIPOLAR,OK
5,ethylenediamine,"ethane-1,2-diamine",InChI=1S/C2H8N2/c3-1-2-4/h1-4H2,NCCN,NCCN,107-15-3,Amines,60.069,3.97041,2.82526,251.777,0.0,0.00021,1045.0,2,2,1.73,284.65,486.65,193,12.49,235,0.15,273.15,389.85,390,0.15,390.0,missing,missing,missing,missing,ASSOCIATIVE,OK
6,"1,2-dibromoethane","1,2-dibromoethane",InChI=1S/C2H4Br2/c3-1-2-4/h1-2H2,BrCCBr,BrCCBr,106-93-4,Halogenated compounds,185.868,2.59098,3.5717,323.703,1.0,0.0,0.0,0,0,1.54,283.15,405.8,129,2.26,134,0.15,283.65,372.15,69,0.15,69.0,missing,missing,missing,missing,DIPOLAR,OK
7,ethyl bromide,bromoethane,"InChI=1S/C2H5Br/c1-2-3/h2H2,1H3",CCBr,CCBr,74-96-4,Halogenated compounds,107.957,2.16801,3.52926,266.03,2.03,0.0,0.0,0,0,1.77,218.0,503.0,172,2.98,176,0.65,161.55,473.0,216,0.67,204.0,0.22,12.0,missing,missing,DIPOLAR,OK
8,"1,2-ethanediol","ethane-1,2-diol","InChI=1S/C2H6O2/c3-1-2-4/h3-4H,1-2H2",OCCO,OCCO,107-21-1,Alcohols,62.037,1.71084,3.73481,354.183,0.0,0.01627,2120.08,2,2,2.16,278.3,570.35,553,5.1,630,2.67,250.2,473.15,1429,2.67,1428.0,0.0,1.0,missing,missing,ASSOCIATIVE,OK
9,ethyl iodide,iodoethane,"InChI=1S/C2H5I/c1-2-3/h2H2,1H3",CCI,CCI,75-03-6,Halogenated compounds,155.944,1.94206,3.82267,319.639,1.91,0.0,0.0,0,0,1.82,173.0,398.15,168,2.92,174,0.22,183.15,343.15,81,0.22,81.0,missing,missing,missing,missing,DIPOLAR,OK


In [18]:
# Info for Clapeyron Database File:
# species Mw segment sigma epsilon dipole n_H n_e source
num_rows = nrow(df)
df2 = DataFrame(
    species = fill(missing, num_rows),
    isomeric_SMILES = fill(missing, num_rows), 
    family = fill(missing, num_rows),
    Mw = fill(missing, num_rows),
    segment = fill(missing, num_rows),
    sigma = fill(missing, num_rows),
    epsilon = fill(missing, num_rows),
    dipole = fill(missing, num_rows),
    kappa_ab = fill(missing, num_rows),
    epsilon_k_ab = fill(missing, num_rows),
    source = fill(missing, num_rows)
) ;


In [19]:
df2.species = df.common_name
df2.Mw = df.molarweight
df2.isomeric_SMILES = df.isomeric_smiles
df2.family = df.family
df2.segment = df.m
df2.sigma = df.sigma
df2.epsilon = df.epsilon_k
df2.dipole = df.mu
df2.kappa_ab = df.kappa_ab
df2.epsilon_k_ab = df.epsilon_k_ab
df2[!, :source] .= "10.1021/acs.iecr.3c02255" ;

In [20]:
filtered_data_chem_family = filter(row -> occursin("Alkane", row.family), df2) ;
filtered_data_chem_family = sort(filtered_data_chem_family, :Mw) ;

#println(filtered_data_chem_family)

In [27]:
filtered_data_chem_family = filter(row -> occursin("nonylphenols", row.common_name), df) 


Row,common_name,iupac_name,inchi,canonical_smiles,isomeric_smiles,cas,family,molarweight,m,sigma,epsilon_k,mu,kappa_ab,epsilon_k_ab,na,nb,mard_psat,t_min_psat,t_max_psat,points_psat,mard_psat_incl_outlier,points_psat_incl_outlier,mard_density,t_min_density,t_max_density,points_density_liquid,mard_density_liquid_single_phase,points_density_liquid_single_phase,mard_density_equi,points_density_equi,mard_density_vapor,points_density_vapor,opt,bounds_violation
,String,String?,String,String,String,String15,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Float64,Int64,Float64,Int64,Float64,Float64,Float64,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,String15,String
1,nonylphenols (mixture of isomeres),2-nonylphenol,"InChI=1S/C15H24O/c1-11(2)9-12(3)10-13(4)14-5-7-15(16)8-6-14/h5-8,11-13,16H,9-10H2,1-4H3",CC(C)CC(C)CC(C)c1ccc(O)cc1,CC(C)CC(C)CC(C)c1ccc(O)cc1,25154-52-3,Aromatic hydrocarbons,220.183,4.39731,4.24914,334.036,0.0,0.0001,200.0,1,1,5.34,436.15,594.96,16,8.8,19,0.63,293.15,423.15,13,0.63,13.0,missing,missing,missing,missing,ASSOCIATIVE,"kappa_ab: lower, epsilon_k_ab: lower"


In [22]:
model = PPCSAFT(["methane"],verbose = true)


[ Info: Searching for assocdata headers ["bondvol", "epsilon_assoc"] for query ["methane"] at /home/mgadaloff/.julia/packages/Clapeyron/0fqKa/database/SAFT/PCSAFT/PPCSAFT/PPCSAFT_assoc.csv ...
[ Info: Searching for singledata headers ["Mw", "segment", "sigma", "epsilon", "dipole", "n_a", "n_b", "type", "bounds_violation"] for query ["methane"] at /home/mgadaloff/.julia/packages/Clapeyron/0fqKa/database/SAFT/PCSAFT/PPCSAFT/PPCSAFT_like.csv ...
┌ Info: Found single component data: Mw with values:
└  methane => 16.031
┌ Info: Found single component data: segment with values:
└  methane => 1.0
┌ Info: Found single component data: sigma with values:
└  methane => 3.70051
┌ Info: Found single component data: epsilon with values:
└  methane => 150.07147
┌ Info: Found single component data: dipole with values:
└  methane => 0.0
┌ Info: Found single component data: n_a with values:
└  methane => 0
┌ Info: Found single component data: n_b with values:
└  methane => 0
┌ Info: Found single compone

PPCSAFT{BasicIdeal} with 1 component:
 "methane"
Contains parameters: Mw, segment, sigma, epsilon, dipole, dipole2, epsilon_assoc, bondvol

In [47]:
fieldnames(typeof(model.params.epsilon_assoc))

println(model.params.epsilon_assoc)


AssocParam{Float64}("epsilon_assoc")Float64[]


In [23]:
function make_model(Mw, m, σ, ϵ, λ_a, λ_r)
    model = PPCSAFT(["methane"])
    
    model.params.Mw[1] = Mw
    model.params.segment[1] = m
    model.params.sigma[1] = σ
    model.params.lambda_a[1] = λ_a
    model.params.lambda_r[1] = λ_r
    model.params.epsilon[1] = ϵ

    return model
end

make_model (generic function with 1 method)

In [24]:
fieldnames(typeof(model6))
fieldnames(typeof(model6.params))
fieldnames(typeof(model6.params.epsilon_assoc.values))
typeof(model6.params.epsilon_assoc.values)

LoadError: UndefVarError: `model6` not defined

In [ ]:
model = PCSAFT(["methanol"],verbose=true)

[ Info: Searching for assocdata headers ["epsilon_assoc", "bondvol"] for query ["methanol"] at /home/mgadaloff/.julia/packages/Clapeyron/0fqKa/database/SAFT/PCSAFT/PCSAFT_assoc.csv ...
┌ Info: Found association component data: epsilon_assoc with values:
└  (methanol,H) ⇋ (methanol, e) => 2899.5
┌ Info: Found association component data: bondvol with values:
└  (methanol,H) ⇋ (methanol, e) => 0.035176
[ Info: Searching for singledata headers ["Mw", "segment", "sigma", "epsilon", "n_H", "n_e"] for query ["methanol"] at /home/mgadaloff/.julia/packages/Clapeyron/0fqKa/database/SAFT/PCSAFT/PCSAFT_like.csv ...
┌ Info: Found single component data: Mw with values:
└  methanol => 32.042
┌ Info: Found single component data: segment with values:
└  methanol => 1.5255
┌ Info: Found single component data: sigma with values:
└  methanol => 3.23
┌ Info: Found single component data: epsilon with values:
└  methanol => 188.9
┌ Info: Found single component data: n_H with values:
└  methanol => 1
┌ Info: 

PCSAFT{BasicIdeal, Float64} with 1 component:
 "methanol"
Contains parameters: Mw, segment, sigma, epsilon, epsilon_assoc, bondvol